In [47]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from scipy.interpolate import InterpolatedUnivariateSpline


In [43]:
# Going to have to define some sort of way of finding the engine speed from the title of the file. #
# For now I will get the rest of the calculation process going
rpm = 2700

In [61]:
# Import the data
data = pd.read_csv("./data/test_export_001.csv")


In [63]:
# rename the columns into something much more comprehensible. 
data.rename(columns={
    "Angle01" : "crank_angle",
    "ptot:exhrunner-1:050":"total_pressure",
    "MASSFLW:exhrunner-1:050":"mass_flow_rate",
    "VEL:exhrunner-1:050":"velocity",
    "TTOT:exhrunner-1:050":"total_temp",
    "HG2:exhrunner-1:050":"UHTE",
    "dynvis:exhrunner-1:050":"dynamic_viscosity",
    "kinvis:exhrunner-1:050":"kinematic_viscosity"
}, inplace = True) 
data.head()


,crank_angle,total_pressure,mass_flow_rate,velocity,total_temp,UHTE,dynamic_viscosity,kinematic_viscosity
0,-94.0,1.649667,0.004789,7.050822,599.34485,1.0,0.000029,0.00003
1,-93.0,1.648886,0.004705,6.928965,599.30475,1.0,0.000029,0.00003
2,-92.0,1.648119,0.004622,6.809809,599.26540,1.0,0.000029,0.00003
3,-91.0,1.647363,0.004541,6.693581,599.22644,1.0,0.000029,0.00003
4,-90.0,1.646615,0.004462,6.579988,599.18774,1.0,0.000029,0.00003


In [57]:
# Now calculate the acceleration for each. 
# find negative start point
val = data.crank_angle[0]

print("First value ", val)

# Shift the first angle to 0 degrees to then convert to time
if val < 0:
    data.crank_angle = data.crank_angle.apply(lambda x: x - val)

# Now use engine speed to convert crank angle to time
rps = rpm/60
seconds_per_rev = 1/rps
data["time"] = data.crank_angle.apply(lambda x: (x/360)*seconds_per_rev)
data.head()

First value  -94.0


,crank_angle,total_pressure,mass_flow_rate,velocity,total_temp,UHTE,dynamic_viscosity,kinematic_viscosity,time
0,0.0,1.649667,0.004789,7.050822,599.34485,1.0,0.000029,0.00003,0.000000
1,1.0,1.648886,0.004705,6.928965,599.30475,1.0,0.000029,0.00003,0.000062
2,2.0,1.648119,0.004622,6.809809,599.26540,1.0,0.000029,0.00003,0.000123
3,3.0,1.647363,0.004541,6.693581,599.22644,1.0,0.000029,0.00003,0.000185
4,4.0,1.646615,0.004462,6.579988,599.18774,1.0,0.000029,0.00003,0.000247


In [58]:
# Calculate the gradient of the graph at each point. 
x = data.time
y = data.velocity
f2 = InterpolatedUnivariateSpline(x, y)
#Get dervative
der = []
for i in range(len(y)):

    h = 1e-4
    der.append( ( f2(x[i]+h)-f2(x[i]-h) )/(2*h) )
der = np.array(der) 

In [59]:
# Add the acceleration to the dataframe
data["acceleration"] = der

# Now convert the crank angle back
if data.crank_angle[0] > -90:
    data.crank_angle = data.crank_angle.apply(lambda x: x + val)
data.head()

,crank_angle,total_pressure,mass_flow_rate,velocity,total_temp,UHTE,dynamic_viscosity,kinematic_viscosity,time,acceleration
0,-94.0,1.649667,0.004789,7.050822,599.34485,1.0,0.000029,0.00003,0.000000,-1991.730154
1,-93.0,1.648886,0.004705,6.928965,599.30475,1.0,0.000029,0.00003,0.000062,-1950.853793
2,-92.0,1.648119,0.004622,6.809809,599.26540,1.0,0.000029,0.00003,0.000123,-1906.694683
3,-91.0,1.647363,0.004541,6.693581,599.22644,1.0,0.000029,0.00003,0.000185,-1863.704850
4,-90.0,1.646615,0.004462,6.579988,599.18774,1.0,0.000029,0.00003,0.000247,-1826.618879


In [60]:
def accel_calc(df, rpm):
    # Now calculate the acceleration for each. 
    # find negative start point
    val = data.crank_angle[0]

    print("First value ", val)

    # Shift the first angle to 0 degrees to then convert to time
    if val < 0:
        data.crank_angle = data.crank_angle.apply(lambda x: x - val)

    # Now use engine speed to convert crank angle to time
    rps = rpm/60
    seconds_per_rev = 1/rps
    data["time"] = data.crank_angle.apply(lambda x: (x/360)*seconds_per_rev)

    # Calculate the gradient of the graph at each point. 
    x = data.time
    y = data.velocity
    f2 = InterpolatedUnivariateSpline(x, y)
    #Get dervative
    der = []
    for i in range(len(y)):

        h = 1e-4
        der.append( ( f2(x[i]+h)-f2(x[i]-h) )/(2*h) )
    der = np.array(der) 

    # Add the acceleration to the dataframe
    data["acceleration"] = der

    # Now convert the crank angle back
    if data.crank_angle[0] > -90:
        data.crank_angle = data.crank_angle.apply(lambda x: x + val)
    return data 

In [64]:
accel_calc(data, 2700)

First value  -94.0


,crank_angle,total_pressure,mass_flow_rate,velocity,total_temp,UHTE,dynamic_viscosity,kinematic_viscosity,time,acceleration
0,-94.00000,1.649667,0.004789,7.050822,599.34485,1.0,0.000029,0.00003,0.000000,-1991.730154
1,-93.00000,1.648886,0.004705,6.928965,599.30475,1.0,0.000029,0.00003,0.000062,-1950.853793
2,-92.00000,1.648119,0.004622,6.809809,599.26540,1.0,0.000029,0.00003,0.000123,-1906.694683
3,-91.00000,1.647363,0.004541,6.693581,599.22644,1.0,0.000029,0.00003,0.000185,-1863.704850
4,-90.00000,1.646615,0.004462,6.579988,599.18774,1.0,0.000029,0.00003,0.000247,-1826.618879
...,...,...,...,...,...,...,...,...,...,...
747,621.99730,1.647710,0.005120,7.549940,599.62024,1.0,0.000029,0.00003,0.044197,-2026.493382
748,622.77800,1.647066,0.005051,7.452167,599.58765,1.0,0.000029,0.00003,0.044246,-2035.247135
749,623.55570,1.646429,0.004983,7.353909,599.55510,1.0,0.000029,0.00003,0.044294,-2033.265273
750,624.31396,1.645813,0.004917,7.258440,599.52356,1.0,0.000029,0.00003,0.044340,-2025.631909
